In [1]:
#!/usr/bin/env python


from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2 import structures
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

from detectron2.evaluation import COCOEvaluator

from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt
import torch
import json
import os
from pathlib import Path

def draw_img(d,dataset_name,save,img_name,temp=""):
    img = cv2.imread(d["file_name"])
    
    print(d["file_name"])
#     print(img)
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(dataset_name), scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(20,20))
    plt.imshow(out.get_image()[:, :, ::-1])
    if save:
        save_dir=os.path.join(os.getcwd(),'imgs_with_bboxes',temp)
        os.makedirs(save_dir,exist_ok=True)

        plt.savefig(os.path.join(save_dir,img_name+"_gt.png"))
    plt.show()
    
def clean_output(outputs,iou_thre=0.4):
    bboxes_pred,pred_scores,pred_classes,_=clean_bboxes(outputs,iou_thre)
    outputs['instances'].pred_boxes=bboxes_pred
    outputs['instances'].pred_classes=pred_classes
    outputs['instances'].scores=pred_scores
    
#     print(len(outputs['instances'].pred_boxes))
    return outputs



def draw_pred(d,dataset_name,save,img_name,predictor,clean_bbox=False,outputs=None):
    file_name=d["file_name"]
    print(file_name)
    im = cv2.imread(file_name)
    
    if outputs is None:
        
        outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
        if clean_bbox:
            if any([True if ann['category_id']==5 else False for ann in d['annotations']]):
                print('__encountering forms do not apply cleaning__')
                pass 
            else:
                outputs=clean_output(outputs)

    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get(dataset_name), 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    plt.figure(figsize=(20,20))
    plt.imshow(out_pred.get_image()[:, :, ::-1])
    if save:
        print('___________saving_____________')
        print(img_name+".png")
        plt.savefig(img_name+".png")
    plt.show()



def convert_format(bbox_lst):
    bboxes_tensor=torch.Tensor(bbox_lst)
    x2y2=bboxes_tensor[:,:2]+bboxes_tensor[:,2::]
    return structures.Boxes(torch.cat((bboxes_tensor[:,:2],x2y2),axis=1))

def load_model(config_path):
    cfg=get_cfg()
    # config_path='/redresearch1/hlai/detectron2/configs/x101_round2_v2.yaml'
    cfg.set_new_allowed(True)

    cfg.merge_from_file(config_path)

    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best_model.pth")  # path to the model we just trained
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
    predictor = DefaultPredictor(cfg)
    print(f'___________model loaded from {config_path}_______________________f')
    return predictor

def filter_boxes(iou_mat):
    n,m=iou_mat.shape
    if m>n:
        dim=m
        axis=1
    else:
        dim=n
        axis=0
    #filter_bool=torch.sum(iou_mat,axis=axis)<=0.2

    ious,box_idx=torch.max(iou_mat,axis=axis)
    filter_bool=np.isin(np.arange(dim),box_idx)

    if m>n:
        iou_mat_subset=iou_mat[:,filter_bool]
    else:
        iou_mat_subset=iou_mat[filter_bool]
    print(iou_mat_subset)
    print(iou_mat_subset.shape)

    assert iou_mat_subset.shape[0]==iou_mat_subset.shape[1]
    problem_bbox_idx=np.arange(dim)[~filter_bool]
    return iou_mat_subset,problem_bbox_idx

    
    
    
def vis_save_missed(missed_lst,save,img_path_prefix,dataset_name,cat_name_arr,predictor,completely_missed=True):
    for i,missed in enumerate(missed_lst):
        print(f'{i}th missed img')
        missed_img,problem_outputs=missed
        problem_cat=problem_outputs['instances'].pred_classes
        missed_cat=cat_name_arr[np.array(problem_cat.to('cpu')).astype(int)]
        print(f'Missing objects : {missed_cat}')
        img_title=str(i)+'_missed_'+ "_".join(missed_cat)
        
        if completely_missed:
            #full predictions
            print('full predictions')
            draw_pred(missed_img,dataset_name,save=save,img_name=img_path_prefix+img_title+'_full_',outputs=None,predictor=predictor,clean_bbox=True)
            #missed bboxes (ground truth)
            print('missed ground truth boxes')
            draw_pred(missed_img,dataset_name,save=save,img_name=img_path_prefix+img_title+'_missed',outputs=problem_outputs,predictor=predictor)
        else:
            #compare ground truth and prediction in one plot
            print('prediction vs ground truth')
            draw_pred(missed_img,dataset_name,save=save,img_name=img_path_prefix+img_title,outputs=problem_outputs,predictor=predictor)
            
       
        print('_________________________________________________________________________________________')

def vis_cls_mistakes(cls_mistakes_imgs,save,img_path_prefix,dataset_name,cat_name_arr,predictor):

    for i,mistakes in enumerate(cls_mistakes_imgs):
        img_title=str(i)
        mistakes_gt_outputs,mistakes_pred_outputs=mistakes[1]
        img=mistakes[0]
#         if len(mistakes_pred):
#             print(f'ground truth classes:{cat_name_arr[np.array(mistakes_gt).astype(int)]},pred classes:{cat_name_arr[np.array(mistakes_pred).astype(int)]}')
#         else:
#             print(f'missed out classes:{cat_name_arr[mistakes_gt]}')
        print('____ground truth_________')
        draw_pred(img,dataset_name,save=save,img_name=img_path_prefix+img_title+'_gt',outputs=mistakes_gt_outputs,predictor=predictor)
        print('__________predicted________')
        draw_pred(img,dataset_name,save=save,img_name=img_path_prefix+img_title+'_pred',outputs=mistakes_pred_outputs,predictor=predictor)
        print('_________________________________________________________________________________________')

def draw_extra_bboxes(labels_to_check,img_path_prefix,dataset_name,save,cat_name_arr,predictor):
    for i,to_check in enumerate(labels_to_check):
        
        img,problem_outputs=to_check
        problem_cat=problem_outputs['instances'].pred_classes
        cat_name=cat_name_arr[np.array(problem_cat.to('cpu')).astype(int)]
        img_title="_".join(cat_name)+str(i)
        print(f'EXTRA CATEGORY PREDICTED:{cat_name}')
        
        draw_img(img,dataset_name,save=save,img_name=img_path_prefix+img_title)
        draw_pred(img,dataset_name,save=save,img_name=img_path_prefix+img_title,outputs=problem_outputs,predictor=predictor)
        
        

def clean_bboxes(outputs,iou_thre=0.4):
    

    bboxes_pred=outputs["instances"].pred_boxes.to('cpu')
    pred_classes=outputs['instances'].pred_classes.to('cpu')
    pred_scores=outputs['instances'].scores.to('cpu')
    
    self_iou=structures.pairwise_iou(bboxes_pred,bboxes_pred)
    clustered_bbox_idx_lst=torch.where((self_iou>=iou_thre)&(self_iou<1))[0]
    if len(clustered_bbox_idx_lst)!=0:
        print('____________cleaning required__________')
#         print(self_iou)
        print(f'before cleaning : {len(bboxes_pred)}')
        bbox_idx_lst=np.arange(len(self_iou))
#         clustered_bboxes=bboxes_pred.tensor[clustered_bbox_idx_lst]
#         remove_idx=clustered_bbox_idx_lst[np.argmin(pred_scores[clustered_bbox_idx_lst])]

        self_iou_subset=self_iou[clustered_bbox_idx_lst]
        candidate_remove_idx=clustered_bbox_idx_lst[torch.argmax(self_iou_subset.sum(axis=1))]
        self_iou_to_check=self_iou[candidate_remove_idx]
        self_iou_sum=self_iou_subset.sum(axis=1)
        if self_iou_sum[0]==self_iou_sum[1]:
            clustered_bboxes=bboxes_pred.tensor[clustered_bbox_idx_lst]
            remove_idx=clustered_bbox_idx_lst[np.argmin(pred_scores[clustered_bbox_idx_lst])]
        else:
            candidate_remove_idx=clustered_bbox_idx_lst[torch.argmax(self_iou_sum)]
            self_iou_to_check=self_iou[candidate_remove_idx]

            if torch.sum((self_iou_to_check>0)&(self_iou_to_check<1))==1:
                remove_idx=clustered_bbox_idx_lst[torch.argmin(self_iou_sum)]
            else:
                remove_idx=candidate_remove_idx


        bboxes_cleaned=bboxes_pred[~np.isin(bbox_idx_lst,remove_idx)]
        scores_cleaned=pred_scores[~np.isin(bbox_idx_lst,remove_idx)]
        classes_cleaned=pred_classes[~np.isin(bbox_idx_lst,remove_idx)]
        print(f'after cleaning : {len(bboxes_cleaned)}')
        
        return bboxes_cleaned,scores_cleaned,classes_cleaned,True
    else:
#         print('_______no cleaning required________')
#         print(self_iou)
        return bboxes_pred,pred_scores,pred_classes,False


def vis_cleaning(imgs_cleaned_lst,save,img_path_prefix,dataset_name,predictor):
    for i,img in enumerate(imgs_cleaned_lst):
        img_title=str(i)
        print('_____no cleaning_________')
        draw_pred(img,dataset_name,save=save,img_name=img_path_prefix+img_title,predictor=predictor)
        img_title=str(i)+'_cleaned'
        print('__________cleaned________')
        draw_pred(img,dataset_name,save=save,img_name=img_path_prefix+img_title,predictor=predictor,clean_bbox=True)
    

def inspect(num_classes,cat_name_arr,iou_thre,dataset_dicts,predictor):
    class_dict={class_id:0 for class_id in range(num_classes)}
    cls_mistakes=[class_dict.copy() for i in range(num_classes)]
    cls_mistakes_imgs=[]
    
    labels_to_check=[]
    completely_missed_lst=[]
    #partially_missed_dict={cat:[] for cat in range(num_classes)}
    partially_missed_lst=[]
    imgs_cleaned_lst=[]
    
#     dataset_dicts_temp=[dataset_dicts[82]]
    for i,img_in in enumerate(dataset_dicts):
        try:
            print(f'_____________________________{i}th image__________________________________________________-')
            file_name=img_in["file_name"]
            print(file_name)
            im = cv2.imread(file_name)
            outputs = predictor(im)
            bbox_lst=[ann['bbox'] for ann in img_in['annotations']]
            class_lst=[ann['category_id'] for ann in img_in['annotations']]
            gt_classes=torch.Tensor(class_lst)
            bboxes_gt=convert_format(bbox_lst)

            bboxes_pred,pred_scores,pred_classes,clean_bool=clean_bboxes(outputs,iou_thre=0.4)

            if clean_bool:
                imgs_cleaned_lst.append(img_in)
    #         n=len(bbox_lst)
    #         m=len(bboxes_pred)

            iou_mat,extra_boxes,extra_classes=check_special_case(bboxes_gt,bboxes_pred,gt_classes,pred_classes)


            if extra_boxes is not None:
                extra_outputs=replace_outputs(outputs,extra_boxes,extra_classes)
                labels_to_check.append([img_in,extra_outputs])
                outputs=predictor(im)

            n,m=iou_mat.shape

    #         iou_mat=structures.pairwise_iou(bboxes_gt,bboxes_pred)


            if n<m:
                print('extra boxes predicted')
                iou_mat_subset,extra_bbox_idx=filter_boxes(iou_mat)

                problem_bboxes=bboxes_pred[extra_bbox_idx]
                problem_classes=pred_classes[extra_bbox_idx]
                problem_outputs=replace_outputs(outputs,problem_bboxes,problem_classes)
                labels_to_check.append([img_in,problem_outputs])

                pred_classes_subset=pred_classes[np.arange(m)[~np.isin(np.arange(m),extra_bbox_idx)]]
                gt_classes_subset=gt_classes

                gt_boxes_subset=bboxes_gt
                pred_boxes_subset=bboxes_pred[np.arange(m)[~np.isin(np.arange(m),extra_bbox_idx)]]



            elif n>m:
                print('boxes missed')
                print(iou_mat)
                print(iou_mat.shape)
                iou_mat_subset,miss_bbox_idx=filter_boxes(iou_mat)
                problem_bboxes=bboxes_gt[miss_bbox_idx]

                problem_classes=gt_classes[miss_bbox_idx]

                problem_outputs=replace_outputs(outputs,problem_bboxes,problem_classes)


                completely_missed_lst.append([img_in,problem_outputs])    
                gt_classes_subset=gt_classes[np.arange(n)[~np.isin(np.arange(n),miss_bbox_idx)]]
                pred_classes_subset=pred_classes

                gt_boxes_subset=bboxes_gt[np.arange(n)[~np.isin(np.arange(n),miss_bbox_idx)]]
                pred_boxes_subset=bboxes_pred

            else:
                print('pred gt dimension matched')
                iou_mat_subset=iou_mat
                pred_classes_subset=pred_classes
                gt_classes_subset=gt_classes
                pred_boxes_subset=bboxes_pred
                gt_boxes_subset=bboxes_gt



            ious,pred_box_idx=torch.max(iou_mat_subset,axis=1)


            pred_classes_subset=pred_classes_subset[pred_box_idx]
            pred_boxes_subset=pred_boxes_subset[pred_box_idx]
            iou_mat_subset=structures.pairwise_iou(gt_boxes_subset,pred_boxes_subset)



            mistakes_gt=gt_classes_subset[gt_classes_subset!=pred_classes_subset]
            mistakes_pred=pred_classes_subset[gt_classes_subset!=pred_classes_subset]
    #         outputs = predictor(im)

            if len(mistakes_gt)!=0:
                print('WRONG PREDICTIONS')
                print(gt_classes_subset,pred_classes_subset)
                print((mistakes_gt,mistakes_pred))

                outputs = predictor(im)
                problem_bboxes_pred=pred_boxes_subset[gt_classes_subset!=pred_classes_subset]
                problem_bboxes_gt=gt_boxes_subset[gt_classes_subset!=pred_classes_subset]

                problem_outputs_gt=replace_outputs(outputs,problem_bboxes_gt.tensor,mistakes_gt)

                outputs = predictor(im)
                problem_outputs_pred=replace_outputs(outputs,problem_bboxes_pred.tensor,mistakes_pred)

                cls_mistakes_imgs.append([img_in,[problem_outputs_gt,problem_outputs_pred]])

                for mistake_gt,mistake_pred in zip(mistakes_gt,mistakes_pred):
                    cls_mistakes[int(mistake_gt)][(int(mistake_pred))]+=1

            ious,box_idx=torch.max(iou_mat_subset,axis=1)
            problem_box_idx=box_idx[ious<=iou_thre]



            if len(problem_box_idx)!=0:
            #         print(iou_mat_subset)
                print('partially missed boxes')
                outputs = predictor(im)     
                problem_bboxes_pred=pred_boxes_subset[problem_box_idx]
                problem_classes_pred=pred_classes_subset[problem_box_idx]


                problem_bboxes_gt=gt_boxes_subset[problem_box_idx]
                problem_classes_gt=gt_classes_subset[problem_box_idx]           


                boxes_to_compare=torch.cat([problem_bboxes_gt.tensor,problem_bboxes_pred.tensor])
                classes_to_compare=torch.cat([problem_classes_gt,problem_classes_pred])

                problem_outputs=replace_outputs(outputs,boxes_to_compare,classes_to_compare)
                partially_missed_lst.append([img_in,problem_outputs])
        except:
                
            pass
            



            
    return cls_mistakes,labels_to_check,completely_missed_lst,partially_missed_lst,cls_mistakes_imgs,imgs_cleaned_lst


def get_cls_mistakes(num_classes,cls_mistakes,save,save_path,cat_name_arr):
    err_dict={}
    for i in range(num_classes):
        cat_name=cat_name_arr[i]
        err_records_dict={cat_name_arr[key]:val for key,val in cls_mistakes[i].items() if val!=0}
        if len(err_records_dict)!=0:
            err_dict[cat_name]=err_records_dict 
    if save:
        os.makedirs(save_path,exist_ok=True)
        with open(save_path+'cls_err.json','w') as handle:
            json.dump(err_dict,handle)
    return err_dict

def replace_outputs(outputs,pred_bboxes_new,pred_classes_new,pred_scores_new=None):
    
    new_outputs=outputs
    new_outputs['instances'].pred_boxes=pred_bboxes_new
    new_outputs['instances'].pred_classes=pred_classes_new.int()
    out_scores=new_outputs['instances'].scores
    len_diff=len(out_scores)-len(pred_bboxes_new)
    if len_diff<0:
        extra_scores=out_scores[-1].repeat(abs(len_diff))
        new_scores=torch.cat([out_scores,extra_scores])
        new_outputs['instances'].scores=new_scores


    return new_outputs


def check_special_case(bboxes_gt,bboxes_pred,gt_classes,pred_classes,iou_thre=0.4):
    

    
    iou_mat=structures.pairwise_iou(bboxes_gt,bboxes_pred)
    if iou_mat.shape[0]==1:
        iou_thre=0.3
    print(iou_mat)
    iou_mat[(iou_mat>0)&(iou_mat<=iou_thre)]=0
    gt_ious,_=torch.max(iou_mat,axis=1)
    pred_ious,_=torch.max(iou_mat,axis=0)
    print(gt_ious)
    print(pred_ious)
    
     
    if (any(gt_ious==0)) & (any(pred_ious==0)):
        print('found both missing boxes and extra boxes')
        n,m=iou_mat.shape
        subset_box_idx=np.arange(m)[pred_ious!=0]
        extra_boxes_idx=np.arange(m)[pred_ious==0]
        extra_boxes=bboxes_pred[extra_boxes_idx].tensor
        extra_classes=pred_classes[extra_boxes_idx]
        
        iou_mat_subset=iou_mat[:,subset_box_idx]
        print(iou_mat_subset.shape)
        
        
        return iou_mat_subset,extra_boxes,extra_classes
              
    else:
        return iou_mat,None,None
        
    


In [2]:
def main(dataset_name,num_classes,cat_name_arr,config_name,iou_thre,save,img_path_prefix,imgs_path,ann_path):
    
    try:
        register_coco_instances(dataset_name, {},ann_path, imgs_path)
        print('registered')
    except:
        pass
    config_path='/redresearch1/hlai/detectron2/configs/'+config_name
    dataset_dicts=DatasetCatalog.get(dataset_name)
    predictor=load_model(config_path)

    cls_mistakes,labels_to_check,completely_missed_lst,partially_missed_lst,cls_mistakes_imgs,imgs_cleaned_lst=inspect(num_classes,\
                                                                          cat_name_arr,iou_thre,dataset_dicts,predictor)
    
    img_path_prefix0=img_path_prefix+config_name[:-5]+"/cleaned_bboxes/"
    os.makedirs(img_path_prefix0,exist_ok=True)

    vis_cleaning(imgs_cleaned_lst,save,img_path_prefix0,dataset_name,predictor)
    
    img_path_prefix1=img_path_prefix+config_name[:-5]+"/completely_missed_bboxes/"
    os.makedirs(img_path_prefix1,exist_ok=True)
    missed_lst=completely_missed_lst
    print('Completely Missed bboxes')
    vis_save_missed(missed_lst,save,img_path_prefix1,dataset_name,cat_name_arr,predictor)
    
    print('Partially Missed bboxes')
    img_path_prefix2=img_path_prefix+config_name[:-5]+"/partially_missed_bboxes/"
    
    os.makedirs(img_path_prefix2,exist_ok=True)
    missed_lst=partially_missed_lst
    vis_save_missed(missed_lst,save,img_path_prefix2,dataset_name,cat_name_arr,predictor,completely_missed=False)

    print('extra bboxes predicted')
    img_path_prefix3=img_path_prefix+config_name[:-5]+"/extra_bboxes_predicted/"
    os.makedirs(img_path_prefix3,exist_ok=True)
    draw_extra_bboxes(labels_to_check,img_path_prefix3,dataset_name,save,cat_name_arr,predictor)
    
    print('cls mistakes')
    
    save_path=img_path_prefix+config_name[:-5]+'/cls_mistakes/'
    err_dict=get_cls_mistakes(num_classes,cls_mistakes,save,save_path,cat_name_arr)
    vis_cls_mistakes(cls_mistakes_imgs,save,save_path,dataset_name,cat_name_arr,predictor)

    
    
    
    return cls_mistakes,labels_to_check,completely_missed_lst,partially_missed_lst,err_dict,imgs_cleaned_lst


In [178]:

config_name_lst=["x101_mode4_v5_ft_v4.yaml","x101_mode3_v4_ft_v4.yaml"]

config_name_lst=['x101_mode3_v3_ft_v2.yaml','x101_mode4_v4_ft_v3.yaml']
config_name_lst=['x101_mode2_v3_ft_v1.yaml','x101_mode4_v4_ft_v2.yaml']
cat_name_arr=np.array(['text','title','list','table','figure','forms','table of content','caption'])

In [ ]:
for config_name in config_name_lst:
    dataset_name="geolay_test_V4"
    num_classes=8
    cat_name_arr=np.array(['text', 'title', 'list', 'table', 'figure', 'forms', 'toc','caption'])
#     config_name='x101_geosynth_v2.yaml'
    iou_thre=0.5
    save=True
    img_path_prefix="/redresearch1/hlai/geolay/error_cases/synth/val_"
    imgs_path='/redresearch1/hlai/geolay/test_v4/images'
    ann_path='/redresearch1/hlai/geolay/test_v4/annotations/instances.json'
    
    cls_mistakes,labels_to_check,completely_missed_lst,partially_missed_lst,err_dict,imgs_cleaned_lst=main(dataset_name,num_classes,cat_name_arr,config_name,iou_thre,save,img_path_prefix,imgs_path,ann_path)


[07/27 20:20:56 d2.data.datasets.coco]: Loaded 120 images in COCO format from /redresearch1/hlai/geolay/test_v4/annotations/instances.json
___________model loaded from /redresearch1/hlai/detectron2/configs/x101_mode2_v3_ft_v1.yaml_______________________f
_____________________________0th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/FA010%20Title%20page_Copyright_Contents-3.jpg
tensor([[0.9706, 0.0000],
        [0.0000, 0.9159]])
tensor([0.9706, 0.9159])
tensor([0.9706, 0.9159])
pred gt dimension matched
_____________________________1th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/FA005%20Vol1%20-%20Title%20page_Copyright_Table%20of%20Contents-6.jpg
tensor([[0.9834]])
tensor([0.9834])
tensor([0.9834])
pred gt dimension matched
_____________________________2th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/EB009%20Title%20page_Content

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9199, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.8995, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8752, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.8075, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.9048, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.7896],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.8158, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8733, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.8804, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

tensor([[0.9574, 0.0000],
        [0.0000, 0.9081]])
tensor([0.9574, 0.9081])
tensor([0.9574, 0.9081])
pred gt dimension matched
_____________________________33th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/91814768_91814769.png
tensor([[0.0000, 0.0632, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.9046, 0.0000],
        [0.0000, 0.0000, 0.8289, 0.0000, 0.0000, 0.0000],
        [0.9753, 0.3003, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2798, 0.8914, 0.0000, 0.0016, 0.0000, 0.0000],
        [0.0000, 0.0035, 0.0000, 0.9088, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9283]])
tensor([0.0000, 0.9046, 0.8289, 0.9753, 0.8914, 0.9088, 0.9283])
tensor([0.9753, 0.8914, 0.8289, 0.9088, 0.9046, 0.9283])
boxes missed
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.9046, 0.0000],
        [0.0000, 0.0000, 0.8289, 0.0000, 0.0000, 0.0000],
 

tensor([[0.0000, 0.9757],
        [0.9456, 0.0000]])
tensor([0.9757, 0.9456])
tensor([0.9456, 0.9757])
pred gt dimension matched
_____________________________53th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/465.jpg
tensor([[0.9440, 0.0000],
        [0.0000, 0.9879]])
tensor([0.9440, 0.9879])
tensor([0.9440, 0.9879])
pred gt dimension matched
_____________________________54th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/464.jpg
tensor([[0.0000, 0.0000, 0.9638, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9022, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.9695, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6571, 0.0000,
         0.0000],
        [0.0000, 0.9589, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0

____________cleaning required__________
before cleaning : 26
after cleaning : 25
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8259, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.9250, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.9542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8995],
       

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0831, 0.9370],
        [0.0000, 0.0000, 0.9731, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9824, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8879, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0416, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9658, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0356, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.6736, 0.0284]])
tensor([0.9370, 0.9731, 0.9824, 0.8879, 0.0000, 0.9658, 0.0000, 0.6736])
tensor([0.8879, 0.9824, 0.9731, 0.9658, 0.6736, 0.9370])
boxes missed
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9370],
        [0.0000, 0.0000, 0.9731, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9824, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8879, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9658, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0

partially missed boxes
_____________________________62th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/167.jpg
tensor([[0.0015, 0.9642],
        [0.7574, 0.0332]])
tensor([0.9642, 0.7574])
tensor([0.7574, 0.9642])
pred gt dimension matched
_____________________________63th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/152.jpg
tensor([[0.8134, 0.0000, 0.0210],
        [0.0000, 0.9315, 0.0000]])
tensor([0.8134, 0.9315])
tensor([0.8134, 0.9315, 0.0000])
extra boxes predicted
tensor([[0.8134, 0.0000],
        [0.0000, 0.9315]])
torch.Size([2, 2])
_____________________________64th image__________________________________________________-
/redresearch1/hlai/geolay/test_v4/images/142.jpg
tensor([[0.0000, 0.0000, 0.9908, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9230, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9609, 0.0000],
        [